Importing useful libraries and importing the drive for the dataset

In [114]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import torch
import torch.nn as nn
import sklearn
import pandas as pd
import io
from sklearn.model_selection import train_test_split

In [115]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/My Drive'
/content/drive/My Drive


Importing the data

In [116]:
pddata = pd.read_csv('GSM1586785_ScrH-12A_Exd_14mer_cg.csv.zip', compression='zip', error_bad_lines=False, skiprows=1)

In [117]:
# transforming into a numpy array
pddata = pddata.to_numpy()
# getting the labels as the last column and neglecting the DNA sequence feature
RelKa = pddata[:, -1]
training_data = pddata[:, 2:-1]

Implementation of Isolation Forest for anomaly detection

In [135]:
from sklearn.ensemble import IsolationForest

random_state = np.random.RandomState(42)
# the contamination parameter is calculated by the program without loss of generality (as written in the paper about IF)
model = IsolationForest(n_estimators=100, max_samples='auto', contamination='auto', random_state=random_state)
model.fit(training_data)

{'bootstrap': False, 'contamination': 'auto', 'max_features': 1.0, 'max_samples': 'auto', 'n_estimators': 100, 'n_jobs': None, 'random_state': RandomState(MT19937) at 0x7F16E7404050, 'verbose': 0, 'warm_start': False}


Getting the anomaly score for each sample

In [136]:
scores = model.decision_function(training_data)
anomaly_score = model.predict(training_data)

In [137]:
outliers_number = 1 / anomaly_score.shape[0] * np.count_nonzero(anomaly_score == -1)
print("Percentage of outliers detected", outliers_number)

Percentage of outliers detected 0.12638058389025678


Deleting the samples which are considered outliers by the Isolation Forest Algorithm

In [138]:
outliers_indices = np.where(anomaly_score == -1)[0]
# we check for debugging how many outliers have indeed a high RelKa value
# print(len(outliers_indices > 110000))
training_data = np.delete(training_data, outliers_indices, axis = 0)

[    88    102    176 ... 170548 170549 170565]


In [139]:
RelKa = np.delete(RelKa, outliers_indices, axis = 0)

Standardization of the data

In [140]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(training_data)
training_data = scaler.transform(training_data)

We classify as "good binding sequences" the ones for which the value of RelKa is above 0.5 (this is kind of a hyperparameter)

In [150]:
RelKa_tilda = np.array([1 if prob > 0.5 else 0 for prob in RelKa])

In [151]:
X_train, X_test, y_train, y_test = train_test_split(training_data, RelKa_tilda, test_size=0.7, random_state=42)

We to compute a classification prediction with Random Forests

In [152]:
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, criterion='entropy')

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

In [153]:
from sklearn import metrics

C = metrics.confusion_matrix(y_test, y_pred)
print(y_test, y_pred)
print("Accuracy:", np.trace(C) / len(y_test))
print("The number of true negatives is:", C[0, 0])
print("The number of false negatives is:", C[1,0])
print("The number of false positives is:", C[0,1])
print("The number of true positives is:", C[1,1])

[0 1 0 ... 0 0 0] [0 0 0 ... 0 0 0]
Accuracy: 0.9900111200582844
The number of true negatives is: 102764
The number of false negatives is: 995
The number of false positives is: 47
The number of true positives is: 510


We see that we have a problem with a large number of false negatives, namely with incorrectly predictions of the negative class

A further insight on the errors has to be done, but probably increasing the threshold 0.5 helps already a lot